### Conver Format to yolo
현재 어노테이션 정보들은 모두 절대좌표로 찍혀 있는 상태이다. 이를 정규화된 상대 좌표로 변환한다.

### Yolo Annotation format
yolo dataset의 경우 이미지 하나에 annotation 정보와 클래스 넘버를 포함하고 있는 txt파일 하나가 매핑되는게 일반적이며, txt파일은 아래와 같이 구성된다.

```
클래스 번호   x좌표(어노테이션 박스 중심점)   y좌표(어노테이션 박스 중심점)    annotation box's width    annotation box's height
```

### AI Hub's annotation file


In [1]:
import os
import json
import shutil
import splitfolders
from glob import glob
from ultralytics import YOLO

In [7]:
data_path = "../training_data"

cnt = {0:0, 5:0, 6:0}
image_filenames = [i.replace("\\", "/") for i in glob("../../training_data/images/*")]
json_filenames = [i.replace("\\", "/") for i in glob("../../training_data/labels/*")]
for json_path, image_path in zip(json_filenames, image_filenames):
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)
    cnt[json_file["annotations"]['disease']] += 1
    file.close()

cnt

{0: 9318, 5: 12832, 6: 7328}

In [8]:
labels_dir = "./new_dataset2/labels"  # yolo format으로 저장될 위치
imgs_dir = "./new_dataset2/images"
cnt = {0:0, 5:0, 6:0}
class_num = {0:0, 5:1, 6:2}  # 0 : 정상(0번) 5 : 검은 썩음병(1번) 6 : 배추노균병(2번)

image_filenames = [i.replace("\\", "/") for i in glob("../../training_data/images/*")]
json_filenames = [i.replace("\\", "/") for i in glob("../../training_data/labels/*")]
for json_path, image_path in zip(json_filenames, image_filenames):
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)

    image_filename = json_file["description"]["image"]
    img_extension = image_filename[image_filename.rfind(".") : ]
    image_width = json_file['description']['width']
    image_height = json_file['description']['height']

    cnt[json_file["annotations"]['disease']] += 1

    if cnt[json_file["annotations"]['disease']] <= 7328:
        bboxes = ""  # 바운딩 박스 좌표를 담을 빈 문자열
        for point in json_file["annotations"]["points"]:  # 모든 바운딩 박스 좌표를 순회하면서
            x_tl, y_tl = point["xtl"], point["ytl"]  # x,y Top-left 좌표
            x_br, y_br = point["xbr"], point["ybr"]  # x,y Bottom-right 좌표

            bounding_box_width = (x_br - x_tl) / image_width  # 바운딩 박스 가로 길이 정규화
            bounding_box_height = (y_br - y_tl) / image_height  # 바운딩 박스 세로 길이 정규화
            center_x = int((x_tl + x_br) / 2)  # 바운딩 박스 중심점의 x좌표
            center_y = int((y_tl + y_br) / 2)  # 바운딩 박스 중심점의 y좌표

            normed_center_x = center_x/image_width  # 정규화된 바운딩 박스 중심점의 x좌표
            normed_center_y = center_y/image_height  # 정규화된 바운딩 박스 중심점의 y좌표
            bbox = f"{class_num[json_file['annotations']['disease']]} {normed_center_x} {normed_center_y} {bounding_box_width} {bounding_box_height}\n"
            bboxes += bbox

        txt_filename = os.path.join(labels_dir, image_filename)
        with open(txt_filename.replace(img_extension, ".txt"), "w", encoding="utf8") as txt:
            txt.write(bboxes)
        shutil.move(image_path, os.path.join(imgs_dir, image_filename))

    file.close()
    txt.close()

In [5]:
from glob import glob
import json

json_filenames = [i.replace("\\", "/") for i in glob("../../training_data/labels/*")]

total_width = 0
total_height = 0
for json_path in json_filenames:
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)
    image_width = json_file['description']['width']
    image_height = json_file['description']['height']

    total_width += image_width
    total_height += image_height

    file.close()

print(f"이미지 평균 가로 길이 : {int(total_width / len(json_filenames))}px")
print(f"이미지 평균 세로 길이 : {int(total_height / len(json_filenames))}px")

이미지 평균 가로 길이 : 3679px
이미지 평균 세로 길이 : 3233px


In [10]:
json_filenames = [i.replace("\\", "/") for i in glob(r"./detection_dataset/*/labels/*/*.json")]

cnt = {0:0, 5:0, 6:0}
for json_path in json_filenames:
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)
    if len(json_file["annotations"]["points"]) >= 1:
        cnt[json_file["annotations"]["disease"]] += 1
    else:
        print(f"thers's no any points. filename is {json_file['description']['image']}")
    file.close()

In [11]:
cnt

{0: 1183, 5: 1760, 6: 592}

In [9]:
splitfolders.ratio("./new_dataset2", "./new_detection2", seed=11, ratio=(0.6, 0.2, 0.2))

Copying files: 43968 files [27:55, 26.24 files/s] 


### V8 training params

|params|default|설명|
|---|---|---|
|model|None|사전에 학습된 모델의 가중치를 사용하거나, 모델 자체를 사용하기 위한 parameter|
|data|None|데이터셋 명세가 적힌 yaml 파일의 경로|
|epochs|100|훈련 횟수|
|time|None|학습 시간에 대한 limit을 설정한다.|
|patience|100|early stop을 위한 매개변수. 해당 횟수만큼 모델에 개선이 없으면 조기종료한다.|
|imgsz|640|이미지의 가로 세로 크기를 조절한다. 학습 정확도와 계산 복잡성에 영향을 직접적으로 미친다.|
|save|True|학습 결과를 best/last.pt로 저장한다.|
|save_preiod|-1|학습 중 checkpoint를 저장할지 말지에 대한 여부. -1은 비활성화를 의미하며, 지정하는 숫자의 epochs마다 저장한다.|
|cache|None|데이터셋 이미지를 메모리 혹은 디스크에 캐싱하게 하여 메모리 사용량은 증가하지만 교육 속도는 향상된다.|

In [ ]:
os.environ['WANDB_DISABLED'] = 'true'

def main():
    model = YOLO("./detection_model/yolov8l.pt", task="detect")
    model.train(
        model = "./detection_model/yolov8l.pt",
        data = "./new_detection2/cabbage_data.yaml",
        imgsz=640,
        epochs=100,
        batch=16,  # batch=-1 -> 장비가 허용하는 범위 내에서 동적으로 설정
        project="detection_model",
        name="cabbage_detection",
        workers=2,
        plots=True,
        device=0,  # gpu 사용
        patience=10,
        translate=0.0,  # 수평 혹은 수직이동
    )

if __name__ == "__main__":
    main()

In [13]:
custom_model = YOLO(r"./detection_model/cabbage_detection2/weights/best.pt")
results = custom_model.predict(source=r"C:\Users\admin\Desktop\testing\*", save=True)


image 1/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0005_S01_1.jpg: 320x640 1 normal, 31.1ms
image 2/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0070_S01_1.jpg: 320x640 1 normal, 29.1ms
image 3/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0072_S01_1.jpg: 320x640 1 normal, 29.1ms
image 4/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0092_S01_1.jpg: 320x640 1 normal, 29.1ms
image 5/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0002.jpeg: 640x640 1 black_rot, 50.7ms
image 6/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0005.jpeg: 640x640 1 black_rot, 168.5ms
image 7/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0007.jpeg: 640x640 1 black_rot, 27.1ms
image 8/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_b0004.jpeg: 640x640 1 black_rot, 27.1ms
image 9/12 C:\Users\ad

In [4]:
from PIL import Image

img = Image.open(r"C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0005_S01_1.jpg")
results = custom_model(img)

0: 320x640 1 normal, 217.9ms
Speed: 0.0ms preprocess, 217.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)


In [15]:
boxes = results[0].boxes
print(len(boxes))
print(f"1. class 번호(0 = 정상, 1 = 검음썩음병, 2 = 배추노균병) {boxes.cls}\n")
print(f"2. 바운딩 박스의 각 꼭짓점의 절대좌표 {boxes.xyxy}\n")
print(f"3. 바운딩 박스의 중심점의 xy좌표, 바운딩 박스의 너비와 높이 {boxes.xywh}\n")
print(f"4. 정규화된 바운딩 박스의 각 꼭짓점의 절대좌표 {boxes.xyxyn}\n")
print(f"5. 정규화된 바운딩 박스의 중심점의 xy좌표, 바운딩 박스의 너비와 높이 {boxes.xywhn}\n")
print(f"6. 해당 객체의 분류 정확도(바운딩 박스 내부에 객체가 있다는 전제 하에, 클래스에 대한 객체의 조건부 확률) {boxes.conf}\n")
print(f"7. 데이터의 함축형. (정규화된 각 꼭짓점 좌표, conf, class_number){boxes.data}\n")

1
1. class 번호(0 = 정상, 1 = 검음썩음병, 2 = 배추노균병) tensor([0.], device='cuda:0')

2. 바운딩 박스의 각 꼭짓점의 절대좌표 tensor([[1131.7333,    0.0000, 3060.8257, 2186.5581]], device='cuda:0')

3. 바운딩 박스의 중심점의 xy좌표, 바운딩 박스의 너비와 높이 tensor([[2096.2795, 1093.2791, 1929.0924, 2186.5581]], device='cuda:0')

4. 정규화된 바운딩 박스의 각 꼭짓점의 절대좌표 tensor([[0.2439, 0.0000, 0.6597, 0.9957]], device='cuda:0')

5. 정규화된 바운딩 박스의 중심점의 xy좌표, 바운딩 박스의 너비와 높이 tensor([[0.4518, 0.4979, 0.4158, 0.9957]], device='cuda:0')

6. 해당 객체의 분류 정확도(바운딩 박스 내부에 객체가 있다는 전제 하에, 클래스에 대한 객체의 조건부 확률) tensor([0.9501], device='cuda:0')

7. 데이터의 함축형. (정규화된 각 꼭짓점 좌표, conf, class_number)tensor([[1.1317e+03, 0.0000e+00, 3.0608e+03, 2.1866e+03, 9.5008e-01, 0.0000e+00]], device='cuda:0')

